# Notes: Generative Models

Containing:
* An introduction to probabilistic models, why we need them, and how to work with them
* Hopefully, some resolution of the frequently confusing key ideas from the overview: (i) data are random, and (2) data are constants

## Models and inference

The basic inference problem we will be concerned with can take several forms, including
* inference about what values of the parameters of a model are prefered by the data
* inference about whether the data prefer one model over another (model selection)
* inference about data we might gather in the future (prediction)

In all cases, being good scientists, we'll want to make statements about how certain we are of one conclusion vs another. As we'll see, this means we will need to get confortable working with probabilistic statements.

But what exactly is this model referred to above? In physics, "model" refers to a mathematical description of some physical system that we're studying. It may be an imperfect description, but by expressing it mathematically we should be able to calculate things and all agree on the answers, given the assumptions.

In statistical analysis, "model" means essentially the same thing. Specifically, we deal in _generative models_, meaning models that include not just the system we're interested in learning about, but also any additional processes required to describe the production (generation) of the observed data. In other words, a generative model is a formal recipe for producing the data we observed, including
* physical processes happening out there in the Universe
* instrumental effects and the measurement process
* any computations done prior to calling the result "the data"

We could equally well think of a generative model as a procedure for producing mock data sets that should be statistically equivalent to the one we observed.

If we do think of model building in the sense of simulating potential data for a moment, the first two bullets above will generally require us to introduce some randomness. Conceptually, there are different reasons why a given step in the data production recipe might include randomness:
* It may be intrinsically random (e.g. quantum mechanical)
* It may be random in the classical statistical mechanics sense, i.e. the system is so complex that its detailed state is unknowable, such that we can only make statistical statements about it
* It may be _effectively_ random, in the sense that there is some important parameter that in reality has a fixed value, but that we simply don't know. [Endnote 1] We could, however, vary its value according to some probability distribution when we generate different mock data sets, thus somehow propagating our ignorance of this parameter to the final result. (This is a generalization of the previous bullet.)

The upshot is that, while we can think of a generative model as a recipe (or psuedo-code) for simulating mock data, many of the steps in that recipe would involve invoking a random number generator. When we do inference with real data, it's the Universe randomizing things (and/or our ignorance making them effectively random). This is the sense behind the first key idea from the overview notes: "All data we collect include some degree of randomness."

### Aside: but, _also_, data are constants

An example will help explain this.

Consider the following **amazing** cartoon of the process of capturing an image of some source (let's say a star) in the sky.

<table><tr width=75%>
<td><img src="graphics/genmod_cartoon.png" alt="Cartoon of starlight being recorded as an image"></td>
</tr></table>

We could begin sketching out a generative model this way:
* There is a star, whose physical model would include things like surface temperature and luminosity.
* Light from the star travels to us, in this case meaning to just in front of our telescope. This part of the model might involve things like extinction and reddening (before reaching the Earth), and absorption and smoothing (in our atmosphere).
* The telescope optics focus the light, imparting some vignetting and abberation on the way. After going through a filter to isolate a certain range of wavelengths, now we're at the focal plane, just above the imaging detector.
* Light hits the detector, is converted to an electronic signal (with some efficiency), and is read out (again, with some efficiency).
* The signal that has been read out is recorded to a hard drive. It is now "the data". (Yes, this ignores unwanted signals like cosmic rays and dark current. We're keeping it simple.)

This is where the symmetry between simulating mock data and doing inference with real data ends. If we're simulating, we can go forth (after specifying the above more concretely in code) and produce as many mock images as we like. If we've done a real observation, we get the image (or images) that we recorded. In that context, the statement that data are constants just says that we define the generative model as including _everything_ up to the point when the data are recorded, in particular including all the steps that involve randomness or uncertainty.

<table><tr width=75%>
<td><img src="graphics/genmod_cartoon2.png" alt="All randomness/uncertainty lives inside the model"></td>
</tr></table>

One way to think about this is that, in the context of inference, the job of the model is to make a prediction for the specific values that comprise the data, not just for some idealized, intermediate quantity (e.g. the "true image" above). Once recorded, the data are fixed, and the only question is whether the model is consistent with being able to produce them.

### (Aside)$^2$: but what about error bars?

... you might reasonably ask at this point. Don't the data have error bars that allow them to be compared with a model for the true image? Isn't that the point of error bars?

The short answer is no, the data do not have error bars. Error bars are a delusion.

The slightly longer answer is yes, often (but not always) it is possible to define an "error bar" and get away with thinking of it as associated with a given datum rather than an uncertainty of the model's prediction. However, the "not always" is key. We will, therefore, stick with the "data are constants" maxim until we can discuss the specific circumstances when this shortcut is justified. Doing so will avoid some common and serious pitfalls, I promise.

To belabor the point slightly more, consider a simplified version of the above example. I've measured the number of occurances of a rare event in my detector in some time (say, photons from a very dim star). I recognize that most of the generative model will boil down to calculating the predicted mean number of photons we expect to detect in that time, with the actual number recorded begin given by the Poisson distribution for that mean.

Say I measured 9 blips. What's the error bar. Well, I remember that error bar usually means "standard deviation", and the standard deviation of the Poisson distribution is the square root of its mean. My best (and only) estimate of the mean is the number that I got, 9. So, $9 \pm 3$.

What if I had measured just 1 blip. Via the same logic, we would call that $1\pm1$. Ok.

And if I had measured zero blips, then... $0\pm0$?

At this point in the hypothetical scenario (which is totally not based on a real story, I swear), I would be scratching my head and realizing that I wasn't sure what question the error bar was supposed to be answering. If the plan was to use the measured number of blips $\pm$ error to estimate the underlying Poisson mean (or some upstream quantity like the star's flux) and its uncertainty, then this last case is clearly nonsense. Measuring zero photons doesn't mean that the star has a flux of zero with absolute certainty, just that it's very dim (and/or the observation was very short, and/or the telescope was very small). Instead of getting caught up assigning fictitious error bars to the data, I should have instead accepted that they are constants, and focused on inferring (with uncertainties) the thing that I actually cared about, the star's flux.

## Specifying generative models

Now that you understand that data are simultaneously random and constant, and do not have error bars, or at least are sick of hearing about it and ready to move on, let's see how to more concretely write down a generative model.

The first two highly recommended steps have already been done above, namely:
* draw a cartoon of how the data are generated
* sketch out in words what is involved in the cartoon

You might recognize these as excellent pieces of advice also frequently given to (and ignored by) introductory physics students. They also apply here. Being older and wiser than an introductory physics student, you will not ignore them.

To avoid getting bogged down in too many astrophysical specifics, let's simplify the example above slightly, while also introducing some notation.

Our measurement is the number of counts in each pixel. (For the uninitiated - the raw form of almost all astronomical data is a number of counts/activations/events recorded by a detector.)
* There's a star emitting light, whose properties are parametrized by $\theta$. (This would include, for example, luminosity, distance from us, and perhaps temperature, metallicity, spectral type, ...)
* From $\theta$, we can determine the average flux falling on a given pixel $k$, $F_k$.
* Given the exposure time of our observation, $T$, $F_k$ determines the average number of counts expected, $\mu_k$. (There are also conversion factors related to collecting area, efficiency, etc. - we'll subsume all of those into $T$, and assume we know exactly what $T$ is.)
* The number of counts measured in a pixel, $N_k$, is a Poisson draw, given the average $\mu_k$.

To be explicit, this is _significantly_ simplified from the more general example above. In particular, the weighty word "determined" appears a couple times. This means we will not be modeling randomness in those steps (think "deterministic").

We can reduce this to shorthand - think of it as statistical pseudo-code, as follows:
* $\theta \sim $ prior
* $F_k \Leftarrow \theta,k$
* $T =$ known
* $\mu_k \Leftarrow F_k,T$
* $N_k \sim \mathrm{Poisson}(\mu_k)$

Here "$\Leftarrow$" indicates a deterministic dependence, and "$\sim$" means "is distributed as" (not "scales like" as it is often used in physics, nor the fuzzy "is kinda sorta like" as it is also sometimes used in physics). Hence, every bullet corresponds to a line of code that we could, in principle, write if we were planning to simulate mock data according to this model, with "$\Leftarrow$" indicating normal function calls and "$\sim$" indicating calls to a random number generator. When we refer to the "probabilistic expressions" associated with a model, we generally mean pseudo-code like this.

I haven't written a specific probability distribution (PDF) in the first line, although we would need to in practice. The right hand side of that line is a distribution encoding what we know about the values of $\theta$ _before_ accounting for what we learn from the data (remember, this is also a recipe for simulating mock data), known as the **prior distribution**. We will discuss this much more in the Bayes Law notes (and beyond), so for now just know that some concrete distribution would need to be specified in practice, without minding the details.

## Visualizing generative models

As you might imagine, this linear psuedo-code representation can become unwieldy for more complex models. A useful complement is a visualization called a **probabilistic graphical model** (PGM) or, less often, a **directed acyclic graph** (DAG). Both names are apt; DAG says what it is, and PGM says what it's for. We'll call these beasts PGMs.

The PGM in statistical inference has a somewhat analogous role to the free-body diagram in introductory mechanics. They're both extrememly helpful visual tools, whose use will avoid any number of pitfalls later on. Likewise, every generation of students assumes that _of course_ it doesn't need to bother with drawing these things, and turns out to be mistaken.

Here's the PGM corresponding to the model above:

<img src="graphics/pgms_pixelcounts.png">

The PGM consists of several components:

* **Nodes** (dots and circles) represent PDFs for parameters
* **Edges** (arrows) represent conditional relationships
* **Plates** (rectangles) represent repeated model components whose contents are conditionally independent

There are multiple types of nodes:

* **Circles** represent a generic PDF. This parameter is a *stochastic* ($\sim$) function of the parameters feeding into it.
* **Points** represent a delta-function PDF. This parameter is a *deterministic* ($\Leftarrow$) function of the parameters feeding into it.
* **Double circles** (or shading) indicate measured data. They are stochastic ($\sim$) in the context of generating mock data, but fixed in the context of parameter inference.

Unfortunately, these specific representations are not universal, but this is the convention we'll stick with.

Notice a few things about the PGM:
* By the time we had broken down our model into a list of expressions, it was described in terms of conditional relationships. This is also reflected in the connectedness and directionality of the PGM. That is, for e.g., $\mu_k$ is directly connected to $T$ and $F_K$, but not to $\theta$ (it is _conditionally independent_ of $\theta$).
* The PGM displays the conditional relationships among the parameters of the model, but is not a substitute for the list of expressions, since it doesn't tell us what each PDF is, specifically.

Because the model formalizes a recipe for producing data from some source, it must have a clear beginning and end. In other words, there can be no way to go backwards or orbit by following the arrows (hence the "directed" and "acyclic" in DAG).

## Interpreting generative models

As well as a recipe for producing data, we can also (more formally) interpret a generative model as encoding the joint probability of all our model parameters and data,

$P(\mathrm{data},\mathrm{params}) = P(\mathrm{params}) P(\mathrm{data}|\mathrm{params})$

This equality is just the definition of **conditional probability** (if this seems unfamiliar, or if you'd like some review, see these [separate notes](essential_probability.ipynb)). More generically, the joint probability of A **and** B is the product of the **marginal probability** of A (the probability of A, _irrespective of B_) and the probability of B given (conditional on) A: $P(A,B) = P(A) P(B|A)$. Symmetrically, $P(A,B)$ is also equal to $P(B)P(A|B)$. 


The particular factorization above is sensible given the way we normally interpret a PGM: there are some parameters, and data are generated conditional on their values. To introduce some terminology, $P(\mathrm{params})$ is the prior distribution (again, much more on that coming later), and $P(\mathrm{data}|\mathrm{params})$, is called the **sampling distribution**.

To be explicit, then, the joint probability encoded in the PGM above would just come from multiplying all the PDFs represented:

$p(\theta,T,\{F_k, \mu_k, N_k\}) = p(\theta)\,p(T) \prod_k P(N_k|\mu_k)\,p(\mu_k|F_k,T)\,p(F_k|\theta)$. [Endnote 2]

Here we can factorize the PDFs associated with each pixel, $k$, because of their conditional independence. That is, all the $F_k$, $\mu_k$ and $N_k$ are not fully independent of each other because the all depend on $\theta$ and $T$ - but, conditioned on $\theta$ and $T$, they are independent (there are no further, direct dependences). The assumption that data points are (conditionally) independent isn't always valid, but it is fairly common, and the resulting factorization is convenient.

Several of the PDFs in the above expression are delta functions, according to our model, so we might be forgiven for simplifying them away. To be maximally pedantic, we will marginalize over the parameters in question. Mathematically, this translates to doing a sum or integral over all possible values of the parameters. We would write this as

$p(\theta,\{N_k\}) = \int dF_k\,d\mu_k\,dT\; p(\theta,T,\{F_k, \mu_k, N_k\})$

$\quad = \int dF_k\,d\mu_k\,dT\; p(\theta)\,p(T) \prod_k P(N_k|\mu_k)\,p(\mu_k|F_k,T)\,p(F_k|\theta)$

Note that once a quantity is marginalized over, it no longer appears as part of the expression on the left hand side.

$p(T)$, $p(\mu_k|F_k,T)$ and $p(F_k|\theta)$ are all delta functions. For example, $p(T) = \delta(T-T_0)$, where $T_0$ is the known value of $T$. If the determininstic relationship "$\mu_k \Leftarrow F_k,T$" is, say, $\mu_k=F_kT$, then $p(\mu_k|F_k,T) = \delta(\mu_k - F_kT)$. $p(F_k|\theta)$ would have a similar form, albeit presumably with a more complex function providing the deterministic value of $F_k$. Since these delta functions are all zero except at the value that makes their arguments vanish, the integrals simplify to

$p(\theta,\{N_k\}) = \underbrace{p(\theta)} ~ \underbrace{\prod_k P\left[N_k|\mu_k(\theta,T)\right]}$,

which is, in simplified form, the prior distribution of $\theta$ multiplied by the sampling distribution for each observation, $N_k$.

Was there a point to all this formalism? Granted, in the context of simulating mock data, it isn't obviously helpful to be able to compute the probability of a specific set of parameter and data values being generated by the model. However, when doing inference from real data, computing such probabilities is the name of the game. The fact that we can read the joint probability of a model, in a sensibly factorized form, off of a PGM will be useful, going forward.

#### Endnotes

1. If this bothers you, consider playing [Frogger](https://en.wikipedia.org/wiki/Frogger) while blindfolded. The game is executing a fully deterministic program, so in reality there is no uncertainty about whether the frog will be smooshed if it hops into the road at a given time. The player is missing this key information, but can still, in principle, make predictions about how likely the frog is to be smooshed at any given time using probabilistic modeling. With repeated experimentation, they could even refine this model to make it more accurate. As far as the player is concerned, whether there is an obstacle in the road is _functionally_ random, despite the underlying mechanics being non-random.

2. We'll follow the convention of using capital $P$ for the probability of a discrete outcome, e.g. $P(N)$ where $N$ is integer-values, or for a generic argument like "data". Probability _densities_ over real-valued variables will get a lower-case $p$. The distinction between capital-$P$ Probability (aka "probability mass") and probability density doesn't matter too often, but occasionally can lead to confusion, so we'll try to keep them straight. (See **probability notes**.)